In [1]:
# keras module for building LSTM 
# https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [97]:
file_path = '/Users/seeni-2328/Documents/Seeni/Datasets/zoho/jayandra/user_sentence.txt'
with open(file_path) as f :
    text = f.readlines()
    

In [98]:
curr_dir = '/Users/seeni-2328/Documents/Seeni/Datasets/kaggle/nyt-comments/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

print(len(all_headlines))
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

885


829

In [5]:
len(text)

7406833

In [99]:
import re
lines = []
beforeLines = []
for t in text :
    beforeLines.append(t)
    t =  re.sub("[^A-Za-z0-9 ]","\n", t)
#     lines.extend(t.split('\n'))
    lines.extend([x.strip().lower() for x in t.split('\n') if len(x.split())>1])


In [100]:
print(beforeLines[0][:100])
print(lines[:7])

Hello... I heard about Free Code Camp via Lifehacker (I think: it was a feedly- feed) So, just going
['i heard about free code camp via lifehacker', 'i think', 'it was a feedly', 'just going through the prescribed steps', 'hello world etc', 'have a mix of different technologies would like to get into development side of things', 'looking for some idea on the average amount of time']


In [58]:
len(lines)

210956

In [101]:
max_sequence_len = max([len(x.split()) for x in lines[:25000]])
max_sequence_len

70

In [102]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 
corpus = [clean_text(x) for x in lines]
corpus[:10]

['i heard about free code camp via lifehacker',
 'i think',
 'it was a feedly',
 'just going through the prescribed steps',
 'hello world etc',
 'have a mix of different technologies would like to get into development side of things',
 'looking for some idea on the average amount of time',
 'subjective and varies by level of commitment',
 'it takes to effectively use curriculum ok',
 'i found the documentation']

In [103]:
tokenizer = Tokenizer()

max_length = 20
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    print(total_words)
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            seq_len = len(n_gram_sequence)
            if seq_len > max_length :
                start = seq_len - max_length
                n_gram_sequence = n_gram_sequence[start:seq_len]
                
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
corpus = corpus[:25000]
inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(total_words)
print(len(inp_sequences))
print(len(corpus))
inp_sequences[:10]


10693
10693
117709
25000


[[2, 466],
 [2, 466, 50],
 [2, 466, 50, 204],
 [2, 466, 50, 204, 66],
 [2, 466, 50, 204, 66, 1635],
 [2, 466, 50, 204, 66, 1635, 699],
 [2, 466, 50, 204, 66, 1635, 699, 3162],
 [2, 37],
 [7, 34],
 [7, 34, 5]]

In [88]:
a = [1,2,3,4,5,6,7,8,9,10]
a[5:8]

[6, 7, 8]

In [104]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    print(input_sequences[:10])
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
print(predictors.shape)
print(label.shape)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    2  466]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    2  466   50]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    2  466   50  204]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    2  466   50  204   66]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     2  466   50  204   66 1635]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    2
   466   50  204   66 1635  699]
 [   0    0    0    0    0    0    0    0    0    0    0    0    2  466
    50  204   66 1635  699 3162]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    2   37]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    7   34]
 [   0    0    0    0    0    0    0    0    0    0    

In [105]:
max_sequence_len

20

In [106]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len)) #2288, 10, 16
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100)) 
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 10)            106930    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10693)             1079993   
Total params: 1,231,323
Trainable params: 1,231,323
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.fit(predictors, label, epochs=150, verbose=5)

Epoch 1/150
Epoch 2/150
Epoch 3/150
Epoch 4/150
Epoch 5/150
Epoch 6/150
Epoch 7/150
Epoch 8/150
Epoch 9/150
Epoch 10/150
Epoch 11/150
Epoch 12/150
Epoch 13/150
Epoch 14/150
Epoch 15/150
Epoch 16/150
Epoch 17/150
Epoch 18/150
Epoch 19/150
Epoch 20/150
Epoch 21/150
Epoch 22/150
Epoch 23/150
Epoch 24/150
Epoch 25/150
Epoch 26/150
Epoch 27/150
Epoch 28/150
Epoch 29/150
Epoch 30/150
Epoch 31/150
Epoch 32/150
Epoch 33/150
Epoch 34/150
Epoch 35/150
Epoch 36/150
Epoch 37/150
Epoch 38/150
Epoch 39/150
Epoch 40/150
Epoch 41/150
Epoch 42/150
Epoch 43/150
Epoch 44/150
Epoch 45/150
Epoch 46/150
Epoch 47/150
Epoch 48/150
Epoch 49/150
Epoch 50/150
Epoch 51/150
Epoch 52/150
Epoch 53/150
Epoch 54/150
Epoch 55/150
Epoch 56/150
Epoch 57/150
Epoch 58/150
Epoch 59/150
Epoch 60/150
Epoch 61/150
Epoch 62/150
Epoch 63/150
Epoch 64/150
Epoch 65/150
Epoch 66/150
Epoch 67/150
Epoch 68/150
Epoch 69/150
Epoch 70/150
Epoch 71/150
Epoch 72/150
Epoch 73/150
Epoch 74/150
Epoch 75/150
Epoch 76/150
Epoch 77/150
Epoch 78

In [123]:
def generate_text(seed_text, next_words, model, max_sequence_len):
#     final_result = seed_text + " :"
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
#         final_result += " "+output_word
    return seed_text.title()

In [127]:
print (generate_text("of", 5, model, max_sequence_len))
print (generate_text("i", 5, model, max_sequence_len))

Of The The The The The
I The The The The The
